# Load LLM

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

# Chatbot that uses pages and summaries

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser

def combine_documents(documents: list[Document]) -> str:
    return "\n\n".join([document.page_content for document in documents])


REPHRASE_SYSTEM_PROMPT = """
<PERSONA>
Eres un especialista resolviendo dudas sobre libros de ficción
</PERSONA>

<TASK>
Tu tarea es refrasear la solicitud del usuario para genera una solicitud refraseada.

- Puedes corregir los errores gramaticales
- Puedes mejorar la semántica y orden léxico de la palabras para un mejor entendimiento
</TASK>
"""

REPHRASE_USER_PROMPT = """{user_request}"""

rephrase_prompt = ChatPromptTemplate([
    SystemMessage(content=REPHRASE_SYSTEM_PROMPT),
    ("user", REPHRASE_USER_PROMPT)
])


QA_SYSTEM_PROMPT = """
<PERSONA>
Eres un especialista resolviendo dudas sobre libros de ficción
</PERSONA>

<TAREA>
Tu tarea es responder la pregunta del usuario.
</TAREA>

<RESTRICCIONES>
- Solo responde la pregunta del usuario tomando como contexto lo provisto en <CONTEXTO>.
</RESTRICCIONES>

<CONTEXTO 1>
{context_1}
</CONTEXTO 1>

<CONTEXTO 2>
{context_2}
</CONTEXTO 2>
"""

QA_USER_PROMPT = """
user question: {user_request}
rephrased user question: {rephrased_request}
"""

qa_prompt = ChatPromptTemplate([
    ("ai", QA_SYSTEM_PROMPT),
    ("user", QA_USER_PROMPT)
])

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")

client = QdrantClient(
    url=url,
    api_key=api_key,
    https=True,
    timeout=300
)

vector_store_page = QdrantVectorStore(
    client=client,
    collection_name="db-book-page",
    embedding=OpenAIEmbeddings(model="text-embedding-ada-002"),
)

vector_store_summarized = QdrantVectorStore(
    client=client,
    collection_name="db-book-summarized",
    embedding=OpenAIEmbeddings(model="text-embedding-ada-002"),
)

def debug(x):
    print(x)
    return x

simple_chatbot = (
    {
        "user_request": itemgetter("user_request"),
        "rephrased_request": rephrase_prompt | llm | StrOutputParser()
    }
    | RunnableLambda(debug)
    | RunnablePassthrough() 
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "context_1": itemgetter("rephrased_request") | vector_store_page.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(combine_documents),
        "context_2": itemgetter("rephrased_request") | vector_store_summarized.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(combine_documents)
    }
    | qa_prompt 
    | llm
    | RunnableLambda(lambda x: x.content)
)

# Chatbot that uses pages, summaries and neighbors

In [24]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/fortaleza-digital.pdf")
pages = loader.load()

filtered_documents = [page for page in pages if len(page.page_content) > 0]

In [ ]:
def extract_pages(documents: list[Document]) -> list[int]:
    pages = []

    for doc in documents:
        print(doc.metadata)
        pages.append(doc.metadata["page"])

    return sorted(list(set(pages)))

def add_neighbors(pages: list[int]) -> list[int]:
    n = 3
    pages_with_neighbors = []
    
    for p in pages:
        index_start = max(p - n, 0)
        index_end = min(p + n, 355)
    
        pages_with_neighbors.extend(list(range(index_start, index_end)))
    
    return sorted(list(set(pages_with_neighbors)))

def get_context_by_number_of_page(pages: list[int]) -> str:
    contexts = []

    for p in pages:
        contexts.append(filtered_documents[p])
    
    print(len(contexts))
    return combine_documents(contexts)
    

simple_chatbot = (
    {
        "user_request": itemgetter("user_request"),
        "rephrased_request": rephrase_prompt | llm | StrOutputParser()
    }
    | RunnablePassthrough() 
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "pages": itemgetter("rephrased_request") | vector_store_page.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(extract_pages) | RunnableLambda(add_neighbors),
    }
    | RunnablePassthrough()
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "context_1": itemgetter("pages") | RunnableLambda(get_context_by_number_of_page),
        "context_2": itemgetter("rephrased_request") | vector_store_summarized.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(combine_documents)

    }
    | qa_prompt 
    | llm
    | RunnableLambda(lambda x: x.content)
)

In [34]:
answer = simple_chatbot.invoke({"user_request": "quien es susan fletcher?"})

{'source': 'data/fortaleza-digital.pdf', 'page': 2, 'page_label': '3', '_id': '64fd3506-ba62-4f1f-8dee-733f6d43321e', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 301, 'page_label': '302', '_id': 'bc520e95-7b34-4b30-9312-144387ad6e10', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 104, 'page_label': '105', '_id': '87bd3f3c-cced-4c43-8ec4-0aeb7dbcda78', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 173, 'page_label': '174', '_id': '00ee8668-28fc-4384-a67f-40617a61b6a2', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 219, 'page_label': '220', '_id': 'a74bd031-dd87-4ef7-97c8-23a8546b87c5', '_collection_name': 'db-book-page'}
29


In [35]:
answer

'Susan Fletcher es un personaje de la novela "La fortaleza digital" de Dan Brown. Es la criptógrafa estrella de la Agencia de Seguridad Nacional (NSA) y trabaja en la Sección de Criptografía. Susan es descrita como una mujer inteligente y atractiva, con un alto coeficiente intelectual. A lo largo de la historia, se enfrenta a un desafío crítico cuando la NSA intercepta un código que ni siquiera su supercomputadora puede descifrar. Además, Susan está emocionalmente involucrada con David Becker, un profesor universitario que también se ve envuelto en la trama.'

# Let's add a friendly answer

In [37]:
from langchain_core.output_parsers.string import StrOutputParser

FRIENDLY_PROMPT = """
Convierta la respuesta provista por el usuario en una respuesta amigable que contenga emojis

user_question: {user_request}
answer: {answer}

friendly answer: 
"""

friendly_prompt = ChatPromptTemplate({("user", FRIENDLY_PROMPT)})

simple_chatbot = (
    {
        "user_request": itemgetter("user_request"),
        "rephrased_request": rephrase_prompt | llm | StrOutputParser(),
    }
    | RunnablePassthrough()
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "pages": (
            itemgetter("rephrased_request")
            | vector_store_page.as_retriever(search_kwargs={"k": 5})
            | RunnableLambda(extract_pages)
            | RunnableLambda(add_neighbors)
        )
    }
    | RunnablePassthrough()
    | {
        "user_request": itemgetter("user_request"),
        "answer": {
            "user_request": itemgetter("user_request"),
            "rephrased_request": itemgetter("rephrased_request"),
            "context_1": (
                itemgetter("pages")
                | RunnableLambda(get_context_by_number_of_page)
            ),
            "context_2": (
                itemgetter("rephrased_request")
                | vector_store_summarized.as_retriever(search_kwargs={"k": 5})
                | RunnableLambda(combine_documents)
            )
        }
        | qa_prompt
        | llm
        | StrOutputParser(),
    }
    | friendly_prompt
    | llm
    | StrOutputParser()
)


In [38]:
simple_chatbot.invoke({"user_request": "quien es susan fletcher?"})

{'source': 'data/fortaleza-digital.pdf', 'page': 2, 'page_label': '3', '_id': '64fd3506-ba62-4f1f-8dee-733f6d43321e', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 301, 'page_label': '302', '_id': 'bc520e95-7b34-4b30-9312-144387ad6e10', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 104, 'page_label': '105', '_id': '87bd3f3c-cced-4c43-8ec4-0aeb7dbcda78', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 173, 'page_label': '174', '_id': '00ee8668-28fc-4384-a67f-40617a61b6a2', '_collection_name': 'db-book-page'}
{'source': 'data/fortaleza-digital.pdf', 'page': 219, 'page_label': '220', '_id': 'a74bd031-dd87-4ef7-97c8-23a8546b87c5', '_collection_name': 'db-book-page'}
29


'¡Hola! 😊 Susan Fletcher es un personaje fascinante de la novela "La fortaleza digital" escrita por Dan Brown. 📚 Es la criptógrafa estrella de la ultrasecreta Agencia de Seguridad Nacional (NSA) y trabaja en la Sección de Criptografía. 🕵️\u200d♀️ Susan es una mujer increíblemente inteligente y atractiva, con un alto coeficiente intelectual. A lo largo de la historia, se enfrenta a un gran desafío cuando la NSA intercepta un código que ni siquiera su supercomputadora puede descifrar. 🔍 Además, Susan tiene una conexión especial con David Becker, un profesor universitario que también se ve envuelto en la emocionante trama. 💑✨'

# Let's generate a step back question

...